# Dataframes

## Parameter tuning

In [68]:
import os
import re
import json
import pandas as pd

In [120]:
# Change
run_name = '200521-092727/'

# Constant
project_path = os.path.dirname(os.path.abspath('.'))
directory_path = '/output/solstorm/'
run_path = project_path + directory_path + run_name
nbr_sims = 5
instance_key = 'Instance'
best_obj_key = 'Best Objective'
avg_obj_key = 'Avg. Objective'
max_time_key = 'Max Time'
min_time_key = 'Min Time'
avg_time_key = 'Avg. Time'
max_iter_key = 'Max Iterations'
min_iter_key = 'Min Iterations'
avg_iter_key = 'Avg. Iterations'

parameter_key = 'parameters'
parameter_one_key = 'removal_lower_percentage'
parameter_two_key = 'removal_upper_percentage'

# instance to [best_obj, acc_obj, max_time, min_time, acc_time, max_iter, min_iter, acc_iter]
instance_to_data = {}
parameter_one_values = set()
parameter_two_values = set()
for file_name in os.listdir(run_path):
    split_name = re.split('_|\.', file_name)
    instance_name = split_name[0]
    is_history = split_name[2] == 'history'
    if is_history:
        with open(run_path + file_name) as file:
            history_json = json.load(file)
        
        parameter_one_value = history_json[parameter_key][parameter_one_key]
        parameter_two_value = history_json[parameter_key][parameter_two_key]
        parameter_one_values.add(parameter_one_value)
        parameter_two_values.add(parameter_two_value)
        
        obj = history_json['best_objective']
        time = history_json['runtime']
        it = history_json['number_of_iterations']

        if instance_name in instance_to_data:
            data = instance_to_data[instance_name]

            if obj < data[0]:
                data[0] = obj
            if time > data[2]:
                data[2] = time
            if time < data[3]:
                data[3] = time
            if it > data[5]:
                data[5] = it
            if it < data[6]:
                data[6] = it
            
            data[1] += obj
            data[4] += time
            data[7] += it
        else:
            instance_to_data[instance_name] = [obj, obj, time, time, time, it, it, it]
            
if len(parameter_one_values) > 1 or len(parameter_two_values) > 1:
    print('Multiple parameter values present in run directory!')

In [123]:
df = pd.DataFrame(columns=[instance_key, best_obj_key, avg_obj_key, max_time_key, min_time_key,
                           avg_time_key, max_iter_key, min_iter_key, avg_iter_key])

for instance in instance_to_data:
    data = instance_to_data[instance]
    best_objective = data[0]
    avg_objective = data[1] / nbr_sims
    max_time = data[2]
    min_time = data[3]
    avg_time = data[4] / nbr_sims
    max_iter = data[5]
    min_iter = data[6]
    avg_iter = data[7] / nbr_sims
    row = pd.Series({instance_key: instance, 
                     best_obj_key: best_objective,
                     avg_obj_key: avg_objective, 
                     max_time_key: max_time,
                     min_time_key: min_time,
                     avg_time_key: avg_time, 
                     max_iter_key: max_iter,
                     min_iter_key: min_iter,
                     avg_iter_key: avg_iter})
    df = df.append(row, ignore_index=True)

# df.sort_values(instance_key)

print(parameter_one_key + '=' + str(parameter_one_values))
print(parameter_two_key + '=' + str(parameter_two_values))

df

removal_lower_percentage={0.05}
removal_upper_percentage={0.15}


,Instance,Best Objective,Avg. Objective,Max Time,Min Time,Avg. Time,Max Iterations,Min Iterations,Avg. Iterations
0,T-13-16-2-1,3579.906875,3607.409444,50.601732,42.652259,45.369373,4999,4999,4999.0
1,T-27-34-5-1,6536.702382,7014.532461,247.249574,196.974097,226.370030,4999,4999,4999.0
2,T-19-22-2-1,8820.089514,8830.311736,69.757972,58.759160,65.888272,4999,4999,4999.0
3,T-9-9-1-1,3144.524063,3144.524063,8.408121,7.377739,8.102371,4999,4999,4999.0
4,T-23-27-3-1,4917.248264,5173.620000,216.606487,129.890840,182.751207,4999,4999,4999.0
5,T-15-17-2-1,4308.749722,4313.592340,48.460386,44.704978,46.306202,4999,4999,4999.0
6,T-11-13-1-1,7999.536563,7999.536563,15.085971,11.168693,13.007679,4999,4999,4999.0
7,T-21-27-3-1,5084.458264,5324.285110,160.016109,137.866801,153.144848,4999,4999,4999.0
8,T-17-21-3-1,4741.053090,4752.422853,45.287267,32.048448,38.888988,4999,4999,4999.0
9,T-25-31-4-1,6691.882104,6885.389665,232.742609,203.624529,213.348812,4999,4999,4999.0
